## Gurobi

In [ ]:
!pip install gurobipy

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
import copy
from operator import itemgetter

from gurobipy import GRB

In [ ]:
# Gurobi WLS license file
# Your credentials are private and should not be shared or copied to public repositories.
# Visit https://license.gurobi.com/manager/doc/overview for more information.
WLSACCESSID="e46cde72-e79f-430a-856c-2a1e2ed7de16"
WLSSECRET="6fff3843-ccb6-46f9-9c7a-2b9b78fe7141"
LICENSEID=2414023

# Create an environment with your WLS license
params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID,
}

env = gp.Env(params=params)

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2414023
Academic license - for non-commercial use only - registered to septiana@student.unimelb.edu.au


# Dataset

## Demand

In [ ]:
# Range of Airports (Cities)
C = range (8)
# 0: Adelaide, 1: Brisbane, 2: Canberra, 3: Darwin, 4: Hobart, 5: Melbourne, 6: Perth, 7: Sydney

DM = {0:[ #Monday, Thursday, and Friday
        [0, 336, 0, 134, 0, 1039, 167, 688],
        [336, 0, 161, 0, 143, 1218, 335, 1570],
        [0, 161, 0, 0, 0, 328, 0, 173],
        [134, 0, 0, 0, 0, 167, 0, 0],
        [0, 143, 0, 0, 0, 510, 0, 345],
        [1039, 1218, 328, 167, 510, 0, 681, 2624],
        [167, 335, 0, 0, 0, 681, 0, 521],
        [688, 1570, 173, 0, 345, 2624, 521, 0]
        ],
      1:[ #Tuesday and Wednessday
        [0, 168, 149, 0, 0, 692, 334, 516],
        [168, 0, 322, 155, 143, 1218, 335, 1396],
        [149, 322, 0, 0, 0, 328, 0, 173],
        [0, 155, 0, 0, 0, 0, 132, 0],
        [0, 143, 0, 0, 0, 510, 0, 172],
        [692, 1218, 328, 0, 510, 0, 681, 2624],
        [334, 335, 0, 132, 0, 681, 0, 521],
        [516, 1396, 173, 0, 172, 2624, 521, 0]
        ],
      5:[ #Saturday
        [0, 336, 149, 0, 0, 346, 167, 344],
        [336, 0, 161, 155, 0, 696, 335, 1047],
        [149, 161, 0, 0, 0, 328, 0, 173],
        [0, 155, 0, 0, 0, 0, 132, 0],
        [0, 0, 0, 0, 0, 170, 0, 172],
        [346, 696, 328, 0, 170, 0, 511, 1924],
        [167, 335, 0, 132, 0, 511, 0, 521],
        [344, 1047, 173, 0, 172, 1924, 521, 0]
        ],
      6:[ #Sunday
        [0, 168, 0, 134, 0, 519, 167, 516],
        [168, 0, 161, 310, 0, 870, 335, 1221],
        [0, 161, 0, 0, 0, 328, 0, 345],
        [134, 310, 0, 0, 0, 167, 0, 0],
        [0, 0, 0, 0, 0, 340, 0, 172],
        [519, 870, 328, 167, 340, 0, 511, 2624],
        [167, 335, 0, 0, 0, 511, 0, 521],
        [516, 1221, 345, 0, 172, 2624, 521, 0]
        ]
     }


## Flight Duration

In [ ]:
# Flight Duration for each leg in minute based on online data
# Added 30 minutes for turn around time

M = 9*60 #Dummy flight duration for leg without demand between two different cities
AFD = [ #Initial flight duration
        [0, 185, 125, 255, M, 110, 250, 145],
        [185, 0, 140, 280, 200, 170, 360, 125],
        [125, 140, 0, M, M, 100, M, 90],
        [255, 280, M, 0, M, 290, 260, M],
        [M, 200, M, M, 0, 105, M, 140],
        [110, 170, 100, 290, 105, 0, 285, 115],
        [250, 360, M, 260, M, 285, 0, 335],
        [145, 125, 90, M, 140, 115, 335,0]]

FD = {}
for i in DM.keys():
    FD[i] = [[M for k in C] for k in C]
for i in DM.keys():
    for k in C:
        for l in C:
            if k == l:
                FD[i][k][l] = 0
            if  DM[i][k][l] != 0:
                FD[i][k][l] = AFD[k][l]
FD

{0: [[0, 185, 540, 255, 540, 110, 250, 145],
  [185, 0, 140, 540, 200, 170, 360, 125],
  [540, 140, 0, 540, 540, 100, 540, 90],
  [255, 540, 540, 0, 540, 290, 540, 540],
  [540, 200, 540, 540, 0, 105, 540, 140],
  [110, 170, 100, 290, 105, 0, 285, 115],
  [250, 360, 540, 540, 540, 285, 0, 335],
  [145, 125, 90, 540, 140, 115, 335, 0]],
 1: [[0, 185, 125, 540, 540, 110, 250, 145],
  [185, 0, 140, 280, 200, 170, 360, 125],
  [125, 140, 0, 540, 540, 100, 540, 90],
  [540, 280, 540, 0, 540, 540, 260, 540],
  [540, 200, 540, 540, 0, 105, 540, 140],
  [110, 170, 100, 540, 105, 0, 285, 115],
  [250, 360, 540, 260, 540, 285, 0, 335],
  [145, 125, 90, 540, 140, 115, 335, 0]],
 5: [[0, 185, 125, 540, 540, 110, 250, 145],
  [185, 0, 140, 280, 540, 170, 360, 125],
  [125, 140, 0, 540, 540, 100, 540, 90],
  [540, 280, 540, 0, 540, 540, 260, 540],
  [540, 540, 540, 540, 0, 105, 540, 140],
  [110, 170, 100, 540, 105, 0, 285, 115],
  [250, 360, 540, 260, 540, 285, 0, 335],
  [145, 125, 90, 540, 140, 1

## Cost

In [ ]:
# Fixed Cost
FC = 13470/365

# Variable Cost
v = 4213 #Variable Cost per 60 minutes
VC = [[0 for k in C] for k in C] #Variable Cost per Leg
for k in C:
    for l in C:
        if AFD[k][l] > 0 and AFD[k][l] < M:
            VC[k][l] = v * (AFD[k][l] - 30) // 60
VC

[[0, 10883, 6670, 15798, 0, 5617, 15447, 8074],
 [10883, 0, 7723, 17554, 11936, 9830, 23171, 6670],
 [6670, 7723, 0, 0, 0, 4915, 0, 4213],
 [15798, 17554, 0, 0, 0, 18256, 16149, 0],
 [0, 11936, 0, 0, 0, 5266, 0, 7723],
 [5617, 9830, 4915, 18256, 5266, 0, 17905, 5968],
 [15447, 23171, 0, 16149, 0, 17905, 0, 21416],
 [8074, 6670, 4213, 0, 7723, 5968, 21416, 0]]

In [ ]:
# Number of seats per leg (Number of seats of a plane)
S = 176
# Daily total maximum working hours
TH = 16 * 60

# Stage 1: Modelling with MIP Model

## Model 1: One Plane Many Cycles for Monday Demand

In [ ]:
# Range of the maximum number of flight
F = range (12)
# Range of planes
N = range (40)
a = 0 #the day 0

m1 = gp.Model(env=env)
m1.setParam(GRB.Param.MIPGap, 0.0001)

# Decision variables
x = m1.addVars([(n,f,k,l) for n in N for f in F for k in C for l in C], vtype=GRB.BINARY, name="x")

# Objective
m1.setObjective(gp.quicksum(gp.quicksum(VC[k][l] * x[n,f,k,l] for f in F) + FC * x[n,0,k,l] for n in N for k in C for l in C), GRB.MINIMIZE)

# Constraints
m1.addConstrs((gp.quicksum(x[n,f,k,l] for k in C for l in C) <= 1 for f in F for n in N), name = '2')
m1.addConstrs((gp.quicksum(x[n,f,k,l] * FD[a][k][l] for f in F for k in C for l in C) <= TH for n in N), name = '3')
m1.addConstrs((gp.quicksum(x[n,f,k,l] * S for f in F for n in N) >= DM[a][k][l] for k in C for l in C), name = '4')
m1.addConstrs((gp.quicksum(x[n,f,k,l] for l in C) >=  gp.quicksum(x[n,f+1,k,l]for l in C) for f in F if f < max(F) and f % 2 == 1 for n in N for k in C), name = '5')
m1.addConstrs((x[n,f,k,k] == 0 for n in N for f in F for k in C), name = '6')
m1.addConstrs((x[n,f,k,l] ==  x[n,f+1,k,l] for f in F if f < max(F) and f % 2 ==0 for n in N for k in C for l in C), name = '7')

# Optimize
m1.optimize()

# Display optimal values of decision variables
for v in m1.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to septiana@student.unimelb.edu.au
Optimize a model with 21384 rows, 30720 columns and 148480 nonzeros
Model fingerprint: 0x9dde9f0f
Variable types: 0 continuous, 30720 integer (30720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 19868 rows and 23480 columns
Presolve time: 0.28s
Presolved: 1516 rows, 7240 columns, 33360 nonzeros
Variable types: 0 continuous, 7240 integer (7240 binary)

Root relaxation: objective 1.451321e+06, 3390 iterations, 0.09 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

In [ ]:
#Objective Value Model 1
print(round(float(m1.ObjVal),0))

1451789.0


In [ ]:
#Count the number of the planes for each home base based on Model 2
var_1 = []
for v in m1.getVars():
    if v.x > 1e-6 and v.varName[-6] == '0':
        var_1.append(int(v.varName[-4]))
CP1 = [0 for k in C]
for k in C:
    CP1[k] = var_1.count(k)
print(CP1) #Planes for each base
print(sum(CP1)) #Total used planes

[3, 5, 1, 2, 2, 7, 6, 6]
32


## Model 2: One Plane No Cycles for Monday Demand

In [ ]:
# Range of the maximum number of flight
F = range (12)
# Range of planes
N = range (30)
a = 0

#Model 2: One Plane One Home Base
m2 = gp.Model(env=env)
m2.setParam(GRB.Param.MIPGap, 0.00015)

# Decision variables
x = m2.addVars([(n,f,k,l) for n in N for f in F for k in C for l in C], vtype=GRB.BINARY, name="x")

# Objective
m2.setObjective(gp.quicksum(gp.quicksum(VC[k][l] * x[n,f,k,l] for f in F) + FC * x[n,0,k,l] for n in N for k in C for l in C), GRB.MINIMIZE )

# Constraints
m2.addConstrs((gp.quicksum(x[n,f,k,l] for k in C for l in C) <= 1 for f in F for n in N), name = '2')
m2.addConstrs((gp.quicksum(x[n,f,k,l] * FD[a][k][l] for f in F for k in C for l in C) <= TH for n in N), name = '3')
m2.addConstrs((gp.quicksum(x[n,f,k,l] * S for f in F if f % 2 == 0 for n in N) + gp.quicksum(x[n,f,l,k] * S for f in F if f % 2 == 1 for n in N) >= DM[a][k][l] for k in C for l in C), name = '8')
m2.addConstrs((gp.quicksum(x[n,f,k,l] for l in C) >=  gp.quicksum(x[n,f+1,k,l]for l in C) for f in F if f < max(F) and f % 2 == 1 for n in N for k in C), name = '9')
m2.addConstrs((gp.quicksum(x[n,f,k,l] for k in C) >=  gp.quicksum(x[n,f+1,k,l]for k in C) for f in F if f < max(F) and f % 2 == 0 for n in N for l in C), name = '10')
m2.addConstrs((x[n,f,k,k] == x[n,f+1,k,k] for n in N for f in F if f < max(F) and f % 2 == 1 for k in C), name = '11')
m2.addConstrs((x[n,0,k,k] == 0 for n in N for k in C), name = '12')
m2.addConstrs((gp.quicksum(x[n,0,k,l] for l in C) ==  gp.quicksum(x[n,5,k,l] for l in C) for n in N for k in C), name = '13')
m2.addConstrs((gp.quicksum(x[n,0,k,l] for l in C) ==  gp.quicksum(x[n,11,k,l] for l in C) for n in N for k in C), name = '14')
m2.addConstrs((gp.quicksum(x[n,0,k,l] for n in N for l in C) <= 10 for k in C), name = '15')
m2.addConstrs((gp.quicksum(x[n,0,k,l] for n in N for l in C) == 0 for k in {2,3,4}), name = '16')
m2.addConstrs((gp.quicksum(x[n,0,k,l] for n in N for l in C) >= 2 for k in set(C) - {2,3,4}), name = '17')

# Optimize
m2.optimize()

# Display optimal values of decision variables
for v in m2.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

Set parameter MIPGap to value 0.00015
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to septiana@student.unimelb.edu.au
Optimize a model with 5030 rows, 23040 columns and 122640 nonzeros
Model fingerprint: 0xc5cdd311
Variable types: 0 continuous, 23040 integer (23040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 1744 rows and 4140 columns
Presolve time: 0.08s
Presolved: 3286 rows, 18900 columns, 91800 nonzeros
Variable types: 0 continuous, 18900 integer (18900 binary)

Root relaxation: objective 1.205962e+06, 12945 iterations, 3.54 seconds (5.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [ ]:
#Objective Value Model 2
obj_val = []
obj_val.append(round(float(m2.ObjVal),0))
obj_val

[1206448.0]

In [ ]:
#change the result from Gurobi to be lists of integer
var_int = []
for v in m2.getVars():
    if v.x > 1e-6:
        var_int.append(v.varName[2:-1].split(','))
my_var = []
for id in var_int:
  new_id = list(np.array(id, dtype=int))
  new_id.append(FD[a][new_id[2]][new_id[3]])
  my_var.append(new_id)
my_var = sorted(my_var, key=lambda x: x[0])
my_var #desired lists of integer

[[0, 0, 5, 7, 115],
 [0, 1, 6, 7, 335],
 [0, 2, 6, 5, 285],
 [0, 3, 5, 5, 0],
 [0, 4, 5, 5, 0],
 [0, 5, 5, 5, 0],
 [0, 6, 5, 5, 0],
 [0, 7, 5, 5, 0],
 [0, 8, 5, 5, 0],
 [0, 9, 4, 5, 105],
 [0, 10, 4, 5, 105],
 [0, 11, 5, 5, 0],
 [1, 0, 5, 0, 110],
 [1, 1, 7, 0, 145],
 [1, 2, 7, 5, 115],
 [1, 3, 5, 5, 0],
 [1, 4, 5, 5, 0],
 [1, 5, 5, 5, 0],
 [1, 6, 5, 5, 0],
 [1, 7, 3, 5, 290],
 [1, 8, 3, 5, 290],
 [1, 9, 5, 5, 0],
 [1, 10, 5, 5, 0],
 [1, 11, 5, 5, 0],
 [2, 0, 7, 5, 115],
 [2, 1, 5, 5, 0],
 [2, 2, 5, 5, 0],
 [2, 3, 5, 5, 0],
 [2, 4, 5, 5, 0],
 [2, 5, 7, 5, 115],
 [2, 6, 7, 5, 115],
 [2, 7, 4, 5, 105],
 [2, 8, 4, 1, 200],
 [2, 9, 5, 1, 170],
 [2, 10, 5, 7, 115],
 [2, 11, 7, 7, 0],
 [3, 0, 0, 6, 250],
 [3, 1, 6, 6, 0],
 [3, 2, 6, 6, 0],
 [3, 3, 1, 6, 360],
 [3, 4, 1, 0, 185],
 [3, 5, 0, 0, 0],
 [3, 6, 0, 0, 0],
 [3, 7, 0, 0, 0],
 [3, 8, 0, 0, 0],
 [3, 9, 0, 0, 0],
 [3, 10, 0, 0, 0],
 [3, 11, 0, 0, 0],
 [5, 0, 5, 1, 170],
 [5, 1, 1, 1, 0],
 [5, 2, 1, 1, 0],
 [5, 3, 5, 1, 170],
 [5, 4, 5, 7

In [ ]:
#Count the number of the planes for each home base based on Model 2
var_2 = []
for v in my_var:
    if v[1] == 0:
        var_2.append(v[2])
CP = [0 for k in C]
for k in C:
    CP[k] = var_2.count(k)
print(CP) #Planes for each base
print(sum(CP)) #Total used planes

[3, 4, 0, 0, 0, 8, 2, 7]
24


In [ ]:
#identifying routes for each plane
def routes(data,N):
    dict_plane = {} #keys: plane numbers, values: lists of legs
    list_leg = []

    for plane in N:
      for id in data:
        if id[0] == plane:
            list_leg.append(id[1:])
      if list_leg != []:
        dict_plane[plane]= list_leg
      list_leg = []

    return dict_plane

routes(my_var,N)

{0: [[0, 5, 7, 115],
  [1, 6, 7, 335],
  [2, 6, 5, 285],
  [3, 5, 5, 0],
  [4, 5, 5, 0],
  [5, 5, 5, 0],
  [6, 5, 5, 0],
  [7, 5, 5, 0],
  [8, 5, 5, 0],
  [9, 4, 5, 105],
  [10, 4, 5, 105],
  [11, 5, 5, 0]],
 1: [[0, 5, 0, 110],
  [1, 7, 0, 145],
  [2, 7, 5, 115],
  [3, 5, 5, 0],
  [4, 5, 5, 0],
  [5, 5, 5, 0],
  [6, 5, 5, 0],
  [7, 3, 5, 290],
  [8, 3, 5, 290],
  [9, 5, 5, 0],
  [10, 5, 5, 0],
  [11, 5, 5, 0]],
 2: [[0, 7, 5, 115],
  [1, 5, 5, 0],
  [2, 5, 5, 0],
  [3, 5, 5, 0],
  [4, 5, 5, 0],
  [5, 7, 5, 115],
  [6, 7, 5, 115],
  [7, 4, 5, 105],
  [8, 4, 1, 200],
  [9, 5, 1, 170],
  [10, 5, 7, 115],
  [11, 7, 7, 0]],
 3: [[0, 0, 6, 250],
  [1, 6, 6, 0],
  [2, 6, 6, 0],
  [3, 1, 6, 360],
  [4, 1, 0, 185],
  [5, 0, 0, 0],
  [6, 0, 0, 0],
  [7, 0, 0, 0],
  [8, 0, 0, 0],
  [9, 0, 0, 0],
  [10, 0, 0, 0],
  [11, 0, 0, 0]],
 5: [[0, 5, 1, 170],
  [1, 1, 1, 0],
  [2, 1, 1, 0],
  [3, 5, 1, 170],
  [4, 5, 7, 115],
  [5, 5, 7, 115],
  [6, 5, 0, 110],
  [7, 0, 0, 0],
  [8, 0, 0, 0],
  [9, 0, 0,

In [ ]:
#changing plane numbers based on the home base and adding shifts

def homebase(data):
    dict_base = {}  #keys: 2 digits plane numbers, first digit: city home base code, second digit: plane number
                    #values: lists of legs
    numb = [0 for k in C]
    for k in C:
        for plane in data.keys():
            if data[plane][0][1] == k:
                dict_base[str(k) + str(numb[k])] = data[plane]
                numb[k] += 1
    return dict_base
homebase(routes(my_var,N))

{'00': [[0, 0, 6, 250],
  [1, 6, 6, 0],
  [2, 6, 6, 0],
  [3, 1, 6, 360],
  [4, 1, 0, 185],
  [5, 0, 0, 0],
  [6, 0, 0, 0],
  [7, 0, 0, 0],
  [8, 0, 0, 0],
  [9, 0, 0, 0],
  [10, 0, 0, 0],
  [11, 0, 0, 0]],
 '01': [[0, 0, 7, 145],
  [1, 0, 7, 145],
  [2, 0, 5, 110],
  [3, 5, 5, 0],
  [4, 5, 5, 0],
  [5, 0, 5, 110],
  [6, 0, 7, 145],
  [7, 5, 7, 115],
  [8, 5, 0, 110],
  [9, 0, 0, 0],
  [10, 0, 0, 0],
  [11, 0, 0, 0]],
 '02': [[0, 0, 3, 255],
  [1, 0, 3, 255],
  [2, 0, 5, 110],
  [3, 7, 5, 115],
  [4, 7, 0, 145],
  [5, 0, 0, 0],
  [6, 0, 0, 0],
  [7, 0, 0, 0],
  [8, 0, 0, 0],
  [9, 0, 0, 0],
  [10, 0, 0, 0],
  [11, 0, 0, 0]],
 '10': [[0, 1, 5, 170],
  [1, 5, 5, 0],
  [2, 5, 5, 0],
  [3, 0, 5, 110],
  [4, 0, 7, 145],
  [5, 1, 7, 125],
  [6, 1, 5, 170],
  [7, 5, 5, 0],
  [8, 5, 5, 0],
  [9, 5, 5, 0],
  [10, 5, 5, 0],
  [11, 1, 5, 170]],
 '11': [[0, 1, 7, 125],
  [1, 6, 7, 335],
  [2, 6, 1, 360],
  [3, 1, 1, 0],
  [4, 1, 1, 0],
  [5, 1, 1, 0],
  [6, 1, 1, 0],
  [7, 1, 1, 0],
  [8, 1, 1, 0]

In [ ]:
#Determining the real flights only (delete the dummy flights)
def realflight(data):
    rf = {} #keys: plane number, values: lists of flights
    index = 0
    new_value = []
    for key in data.keys():
        for k in F:
            if data[key][k][-1] != 0:
                new_value.append([index] + data[key][k])
                index += 1
        rf[key] = new_value
        new_value = []
        index = 0
    return rf
real_flight = realflight(homebase(routes(my_var,N)))
real_flight

{'00': [[0, 0, 0, 6, 250], [1, 3, 1, 6, 360], [2, 4, 1, 0, 185]],
 '01': [[0, 0, 0, 7, 145],
  [1, 1, 0, 7, 145],
  [2, 2, 0, 5, 110],
  [3, 5, 0, 5, 110],
  [4, 6, 0, 7, 145],
  [5, 7, 5, 7, 115],
  [6, 8, 5, 0, 110]],
 '02': [[0, 0, 0, 3, 255],
  [1, 1, 0, 3, 255],
  [2, 2, 0, 5, 110],
  [3, 3, 7, 5, 115],
  [4, 4, 7, 0, 145]],
 '10': [[0, 0, 1, 5, 170],
  [1, 3, 0, 5, 110],
  [2, 4, 0, 7, 145],
  [3, 5, 1, 7, 125],
  [4, 6, 1, 5, 170],
  [5, 11, 1, 5, 170]],
 '11': [[0, 0, 1, 7, 125], [1, 1, 6, 7, 335], [2, 2, 6, 1, 360]],
 '12': [[0, 0, 1, 7, 125],
  [1, 3, 1, 7, 125],
  [2, 4, 1, 5, 170],
  [3, 5, 1, 5, 170],
  [4, 6, 1, 7, 125],
  [5, 7, 2, 7, 90],
  [6, 8, 2, 1, 140]],
 '13': [[0, 0, 1, 5, 170],
  [1, 3, 7, 5, 115],
  [2, 4, 7, 0, 145],
  [3, 5, 1, 0, 185],
  [4, 6, 1, 5, 170],
  [5, 11, 1, 5, 170]],
 '50': [[0, 0, 5, 7, 115],
  [1, 1, 6, 7, 335],
  [2, 2, 6, 5, 285],
  [3, 9, 4, 5, 105],
  [4, 10, 4, 5, 105]],
 '51': [[0, 0, 5, 0, 110],
  [1, 1, 7, 0, 145],
  [2, 2, 7, 5, 115],

# Stage 2: Earliness and Tardiness

## Function 1: Calculate the Earliness and Tardiness based on putting the due date between two sequence flights

In [ ]:
def myET_1(data):

    time = [r[-1] for r in data[:-1]]
    list_ET = []

    for k in range (len(time) + 1): #Calculating the Earliness and Tardiness (ET)
        E = 0
        T = 0
        PE = 0
        PT = data[-1][-1]
        for l in range (len(time)):
            if l < k:
                E += (l + 1) * time[l]
                PE += time[l]
            if l >= k:
                T += (len(time[k:])-(l-k))*time[l]
                PT += time[l]
        if PE > 8 * 60 or PT > 8 * 60:
            ET = 10000 #Big M Value
        else:
            ET = E + T
        list_ET.append(ET)

    OT = list_ET.index(min(list_ET)) #The index of the best minimise ET
    DT = [0 for k in range (len(time)+1)]

    for k in range (len(time) + 1): #Calculating the hour of flight
        if k < OT:
            DT[k] = 780 - sum(r[-1] for r in data[k:OT])
        if k >= OT:
            DT[k] = 780 + sum(r[-1] for r in data[OT:k])

    DT_hour = ["{0:0=2d}".format(k // 60)+':'+"{0:0=2d}".format(k % 60) for k in DT] #list of departure time

    last_arrival = DT[-1] + data[-1][-1]
    FT_hour = "{0:0=2d}".format(last_arrival // 60) + ':' + "{0:0=2d}".format(last_arrival % 60) #the last arrival time

    return(DT_hour, FT_hour, min(list_ET)) #Output: departure time, last arrival time, best Earliness/Tardiness

In [ ]:
#Example output of Function 1
myET_1(real_flight['52'])

(['07:20', '10:10', '13:00', '14:55', '16:50', '18:40'], '20:30', 1195)

## Function 2: Calculate the Earliness and Tardiness based on putting the first departure flight at 05:00 or the last arrival flight at 21:00

In [ ]:
def myET_2(data):
    ET_list1 = [] #Scheduling the plane from the first flight
    ET_list2 = [] #Scheduling the plane from the last flight

    time1 = [r[-1] for r in data]
    for k in range(len(time1)):
        value = abs(480-sum(time1[:k]))
        ET_list1.append(value)
    ET_list1

    time2 = time1[::-1]
    for k in range(len(time2)):
        value = abs(480-sum(time2[:k+1]))
        ET_list2.append(value)
    ET_list2.reverse()

    DT = [0 for k in range(len(time1))]
    if sum(ET_list1) < sum(ET_list2):
        for k in range(len(time1)):
            DT[k] = 300 + sum(time1[:k])
    else:
        for k in range(len(time2)):
            DT[k] = 1260 - sum(time2[:k+1])
        DT.reverse()

    DT_hour = ["{0:0=2d}".format(k // 60)+':'+"{0:0=2d}".format(k % 60) for k in DT] #list of departure time

    last_arrival = DT[-1] + time1[-1]
    FT_hour = "{0:0=2d}".format(last_arrival // 60) + ':' + "{0:0=2d}".format(last_arrival % 60) #the last arrival time

    return(DT_hour, FT_hour, min(sum(ET_list1),sum(ET_list2)))

In [ ]:
#Example output of Function 2
myET_2(real_flight['52'])

(['05:00', '07:50', '10:40', '12:35', '14:30', '16:20'], '18:10', 1245)

## Function 3: Compare Function 1 and 2 and choose the best

In [ ]:
def myET(data1,data2):
    mybest = ()
    if data1[-1] < data2[-1]:
        mybest = data1
    else:
        mybest = data2
    return mybest

In [ ]:
data1 = myET_1(real_flight['52'])
data2 = myET_2(real_flight['52'])
myET(data1,data2)

(['07:20', '10:10', '13:00', '14:55', '16:50', '18:40'], '20:30', 1195)

In [ ]:
#Adding the result of the Function 3 to flight dictionary
def updateflight(data):
    uf = {}
    new_value = []
    for key in data.keys():
        mydata = myET(myET_1(data[key]),myET_2(data[key]))
        for k in range(len(mydata[0])):
            new_value.append(data[key][k]+[mydata[0][k]])
        uf[key] = new_value
        new_value = []
    return uf

time_flight = {}
time_flight[0] = updateflight(real_flight)
time_flight

{0: {'00': [[0, 0, 0, 6, 250, '07:45'],
   [1, 3, 1, 6, 360, '11:55'],
   [2, 4, 1, 0, 185, '17:55']],
  '01': [[0, 0, 0, 7, 145, '06:20'],
   [1, 1, 0, 7, 145, '08:45'],
   [2, 2, 0, 5, 110, '11:10'],
   [3, 5, 0, 5, 110, '13:00'],
   [4, 6, 0, 7, 145, '14:50'],
   [5, 7, 5, 7, 115, '17:15'],
   [6, 8, 5, 0, 110, '19:10']],
  '02': [[0, 0, 0, 3, 255, '05:00'],
   [1, 1, 0, 3, 255, '09:15'],
   [2, 2, 0, 5, 110, '13:30'],
   [3, 3, 7, 5, 115, '15:20'],
   [4, 4, 7, 0, 145, '17:15']],
  '10': [[0, 0, 1, 5, 170, '06:10'],
   [1, 3, 0, 5, 110, '09:00'],
   [2, 4, 0, 7, 145, '10:50'],
   [3, 5, 1, 7, 125, '13:15'],
   [4, 6, 1, 5, 170, '15:20'],
   [5, 11, 1, 5, 170, '18:10']],
  '11': [[0, 0, 1, 7, 125, '07:20'],
   [1, 1, 6, 7, 335, '09:25'],
   [2, 2, 6, 1, 360, '15:00']],
  '12': [[0, 0, 1, 7, 125, '05:15'],
   [1, 3, 1, 7, 125, '07:20'],
   [2, 4, 1, 5, 170, '09:25'],
   [3, 5, 1, 5, 170, '12:15'],
   [4, 6, 1, 7, 125, '15:05'],
   [5, 7, 2, 7, 90, '17:10'],
   [6, 8, 2, 1, 140, '18:4

In [ ]:
#Available shifts based on the homebase
def shift_dict(data,d):
    shift_dict = {} #keys: cities, values: lists of [day, plane number, flight duration (minutes), and lists of routes]
    shifts = []
    for k in C:
        shift_dict[k] = []
    for key in data.keys():
        value1 = []
        value2 = []
        total_dur1 = 0
        total_dur2 = 0
        for i in range(len(data[key])):
            if data[key][i][1] < 6: #6 is just len(F)// 2
                total_dur1 += data[key][i][-2]
                value1 += [[data[key][i][0]] + data[key][i][2:4] + [data[key][i][-1]]]
            else:
                total_dur2 += data[key][i][-2]
                value2 += [[data[key][i][0]] + data[key][i][2:4] + [data[key][i][-1]]]

        value1 = [d] + [key] + [total_dur1] + [value1]
        value2 = [d] + [key] + [total_dur2] + [value2]
        if value1[2] != 0:
            shift_dict[int(key[0])] += [value1]
        if value2[2] != 0:
            shift_dict[int(key[0])] += [value2]

    return shift_dict

In [ ]:
totalshifts = shift_dict(time_flight[0],0)

totalshifts

{0: [[0,
   '00',
   795,
   [[0, 0, 6, '07:45'], [1, 1, 6, '11:55'], [2, 1, 0, '17:55']]],
  [0,
   '01',
   510,
   [[0, 0, 7, '06:20'],
    [1, 0, 7, '08:45'],
    [2, 0, 5, '11:10'],
    [3, 0, 5, '13:00']]],
  [0, '01', 370, [[4, 0, 7, '14:50'], [5, 5, 7, '17:15'], [6, 5, 0, '19:10']]],
  [0,
   '02',
   880,
   [[0, 0, 3, '05:00'],
    [1, 0, 3, '09:15'],
    [2, 0, 5, '13:30'],
    [3, 7, 5, '15:20'],
    [4, 7, 0, '17:15']]]],
 1: [[0,
   '10',
   550,
   [[0, 1, 5, '06:10'],
    [1, 0, 5, '09:00'],
    [2, 0, 7, '10:50'],
    [3, 1, 7, '13:15']]],
  [0, '10', 340, [[4, 1, 5, '15:20'], [5, 1, 5, '18:10']]],
  [0, '11', 820, [[0, 1, 7, '07:20'], [1, 6, 7, '09:25'], [2, 6, 1, '15:00']]],
  [0,
   '12',
   590,
   [[0, 1, 7, '05:15'],
    [1, 1, 7, '07:20'],
    [2, 1, 5, '09:25'],
    [3, 1, 5, '12:15']]],
  [0, '12', 355, [[4, 1, 7, '15:05'], [5, 2, 7, '17:10'], [6, 2, 1, '18:40']]],
  [0,
   '13',
   615,
   [[0, 1, 5, '05:05'],
    [1, 7, 5, '07:55'],
    [2, 7, 0, '09:50'],
 

## Model 2 and Earliness/ Tardiness for other demand day

In [ ]:
# Range of the maximum number of flight
F = range (12)
# Range of planes
N = range (30)

for a in {1,5,6}:
    m2a = gp.Model(env=env)
    m2a.setParam("MIPGap", 0.01)

    # Decision variables
    x = m2a.addVars([(n,f,k,l) for n in N for f in F for k in C for l in C], vtype=GRB.BINARY, name="x")

    # Objective
    m2a.setObjective(gp.quicksum(gp.quicksum(VC[k][l] * x[n,f,k,l] for f in F) + FC * x[n,0,k,l] for n in N for k in C for l in C), GRB.MINIMIZE )

    # Constraints
    m2a.addConstrs((gp.quicksum(x[n,f,k,l] for k in C for l in C) <= 1 for f in F for n in N), name = '2')
    m2a.addConstrs((gp.quicksum(x[n,f,k,l] * FD[a][k][l] for f in F for k in C for l in C) <= TH for n in N), name = '3')
    m2a.addConstrs((gp.quicksum(x[n,f,k,l] * S for f in F if f % 2 == 0 for n in N) + gp.quicksum(x[n,f,l,k] * S for f in F if f % 2 == 1 for n in N) >= DM[a][k][l] for k in C for l in C), name = '8')
    m2a.addConstrs((gp.quicksum(x[n,f,k,l] for l in C) >=  gp.quicksum(x[n,f+1,k,l]for l in C) for f in F if f < max(F) and f % 2 == 1 for n in N for k in C), name = '9')
    m2a.addConstrs((gp.quicksum(x[n,f,k,l] for k in C) >=  gp.quicksum(x[n,f+1,k,l]for k in C) for f in F if f < max(F) and f % 2 == 0 for n in N for l in C), name = '10')
    m2a.addConstrs((x[n,f,k,k] == x[n,f+1,k,k] for n in N for f in F if f < max(F) and f % 2 == 1 for k in C), name = '11')
    m2a.addConstrs((x[n,0,k,k] == 0 for n in N for k in C), name = '12')
    m2a.addConstrs((gp.quicksum(x[n,0,k,l] for l in C) ==  gp.quicksum(x[n,5,k,l] for l in C) for n in N for k in C), name = '13')
    m2a.addConstrs((gp.quicksum(x[n,0,k,l] for l in C) ==  gp.quicksum(x[n,11,k,l] for l in C) for n in N for k in C), name = '14')
    # Adding constraint based on Model 2
    m2a.addConstrs((gp.quicksum(x[n,0,k,l] for n in N for l in C) <= CP[k] for k in C), name = '15a')

    # Optimize
    m2a.optimize()

    #Objective Value
    obj_val.append(round(float(m2a.ObjVal),0))

    #change the result from Gurobi to be list of integer
    var_int = []
    for v in m2a.getVars():
        if v.x > 0.9 and v.x < 1.1:
            var_int.append(v.varName[2:-1].split(','))
    my_var = []
    for id in var_int:
      new_id = list(np.array(id, dtype=int))
      new_id.append(FD[a][new_id[2]][new_id[3]])
      my_var.append(new_id)
    my_var = sorted(my_var, key=lambda x: x[0])

    #Determining the real flights only adding the time flights based on ET
    real_flight = realflight(homebase(routes(my_var,N)))
    time_flight[a] = updateflight(real_flight)

    #Determining the total shifts
    for key in totalshifts.keys():
        totalshifts[key] += shift_dict(time_flight[a],a)[key]


Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to septiana@student.unimelb.edu.au
Optimize a model with 5022 rows, 23040 columns and 120720 nonzeros
Model fingerprint: 0x920a59c0
Variable types: 0 continuous, 23040 integer (23040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 1739 rows and 4140 columns
Presolve time: 0.13s
Presolved: 3283 rows, 18900 columns, 91350 nonzeros
Variable types: 0 continuous, 18900 integer (18900 binary)

Root relaxation: objective 1.175753e+06, 22388 iterations, 1.90 seconds (2.73 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

H  688   748                    898696.27397 898627.463  0.01%  1470  154s

Cutting planes:
  Cover: 21
  Implied bound: 30
  MIR: 34
  StrongCG: 23
  GUB cover: 4
  Inf proof: 2
  Zero half: 7
  RLT: 3

Explored 1183 nodes (1666718 simplex iterations) in 154.05 seconds (219.71 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 898696 

Optimal solution found (tolerance 1.00e-02)
Best objective 8.986962739726e+05, best bound 8.986274631849e+05, gap 0.0077%
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to septiana@student.unimelb.edu.au
Optimize a model with 5022 rows, 23040 columns and 120720 nonzeros
Model fingerprint: 0x516f45cf
Variable types: 0 continuous, 23040 integer (23040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  O

In [ ]:
time_flight

{0: {'00': [[0, 0, 0, 6, 250, '07:45'],
   [1, 3, 1, 6, 360, '11:55'],
   [2, 4, 1, 0, 185, '17:55']],
  '01': [[0, 0, 0, 7, 145, '06:20'],
   [1, 1, 0, 7, 145, '08:45'],
   [2, 2, 0, 5, 110, '11:10'],
   [3, 5, 0, 5, 110, '13:00'],
   [4, 6, 0, 7, 145, '14:50'],
   [5, 7, 5, 7, 115, '17:15'],
   [6, 8, 5, 0, 110, '19:10']],
  '02': [[0, 0, 0, 3, 255, '05:00'],
   [1, 1, 0, 3, 255, '09:15'],
   [2, 2, 0, 5, 110, '13:30'],
   [3, 3, 7, 5, 115, '15:20'],
   [4, 4, 7, 0, 145, '17:15']],
  '10': [[0, 0, 1, 5, 170, '06:10'],
   [1, 3, 0, 5, 110, '09:00'],
   [2, 4, 0, 7, 145, '10:50'],
   [3, 5, 1, 7, 125, '13:15'],
   [4, 6, 1, 5, 170, '15:20'],
   [5, 11, 1, 5, 170, '18:10']],
  '11': [[0, 0, 1, 7, 125, '07:20'],
   [1, 1, 6, 7, 335, '09:25'],
   [2, 2, 6, 1, 360, '15:00']],
  '12': [[0, 0, 1, 7, 125, '05:15'],
   [1, 3, 1, 7, 125, '07:20'],
   [2, 4, 1, 5, 170, '09:25'],
   [3, 5, 1, 5, 170, '12:15'],
   [4, 6, 1, 7, 125, '15:05'],
   [5, 7, 2, 7, 90, '17:10'],
   [6, 8, 2, 1, 140, '18:4

In [ ]:
obj_val #Objective value

[1206448.0, 1176221.0, 898696.0, 1061740.0]

## Daily schedule for every city

In [ ]:
#Daily schedule for every city
sched_daily = {} #keys: day, values: dictionary of Value
for day in time_flight.keys():
  Value = {} #keys: cities, value: lists of [destination, plane number, departure time]
  for k in C:
    Value[k] = []
    for plane in time_flight[day].keys():
      for shift in time_flight[day][plane]:
        if shift[0] % 2 == 0:
          if k == shift[2]:
            Value[k] += [[shift[3]] + [plane] + [shift[-1]]]
        else:
          if k == shift[3]:
            Value[k] += [[shift[2]] + [plane] + [shift[-1]]]
    Value[k] = sorted(Value[k], key=itemgetter(-1,0))
  sched_daily[day] = Value

In [ ]:
sched_daily

{0: {0: [[3, '02', '05:00'],
   [7, '01', '06:20'],
   [7, '51', '07:00'],
   [6, '00', '07:45'],
   [1, '60', '09:10'],
   [7, '10', '10:50'],
   [5, '01', '11:10'],
   [1, '13', '12:15'],
   [5, '53', '12:20'],
   [5, '02', '13:30'],
   [5, '61', '14:25'],
   [7, '01', '14:50'],
   [5, '52', '18:40'],
   [5, '54', '19:05']],
  1: [[5, '13', '05:05'],
   [7, '12', '05:15'],
   [5, '10', '06:10'],
   [7, '73', '07:05'],
   [7, '11', '07:20'],
   [0, '53', '09:15'],
   [5, '12', '09:25'],
   [6, '75', '09:25'],
   [4, '55', '09:40'],
   [5, '52', '10:10'],
   [6, '57', '10:15'],
   [7, '76', '10:55'],
   [2, '60', '12:15'],
   [7, '12', '15:05'],
   [7, '73', '15:05'],
   [7, '74', '15:05'],
   [7, '76', '15:05'],
   [5, '10', '15:20'],
   [5, '13', '15:20'],
   [5, '70', '16:15'],
   [7, '53', '17:00'],
   [0, '00', '17:55']],
  2: [[7, '57', '06:40'],
   [5, '60', '14:35'],
   [1, '12', '18:40'],
   [5, '56', '19:20']],
  3: [[0, '02', '09:15'], [5, '51', '16:10']],
  4: [[7, '71', '0

In [ ]:
for key in totalshifts.keys():
    for day in [3,4]:
        totalshifts[key] += shift_dict(time_flight[0],day)[key] #Adding Thursday (3) and Friday (4)
    totalshifts[key] += shift_dict(time_flight[1],2)[key] #Adding Wednessday (2)
totalshifts

{0: [[0,
   '00',
   795,
   [[0, 0, 6, '07:45'], [1, 1, 6, '11:55'], [2, 1, 0, '17:55']]],
  [0,
   '01',
   510,
   [[0, 0, 7, '06:20'],
    [1, 0, 7, '08:45'],
    [2, 0, 5, '11:10'],
    [3, 0, 5, '13:00']]],
  [0, '01', 370, [[4, 0, 7, '14:50'], [5, 5, 7, '17:15'], [6, 5, 0, '19:10']]],
  [0,
   '02',
   880,
   [[0, 0, 3, '05:00'],
    [1, 0, 3, '09:15'],
    [2, 0, 5, '13:30'],
    [3, 7, 5, '15:20'],
    [4, 7, 0, '17:15']]],
  [1, '00', 290, [[0, 0, 7, '05:25'], [1, 0, 7, '07:50']]],
  [1, '00', 645, [[2, 0, 6, '10:15'], [3, 5, 6, '14:25'], [4, 5, 0, '19:10']]],
  [1,
   '01',
   550,
   [[0, 0, 5, '06:15'],
    [1, 1, 5, '08:05'],
    [2, 1, 7, '10:55'],
    [3, 0, 7, '13:00']]],
  [1, '01', 335, [[4, 0, 5, '15:25'], [5, 2, 5, '17:15'], [6, 2, 0, '18:55']]],
  [5, '00', 645, [[0, 0, 6, '08:50'], [1, 5, 6, '13:00'], [2, 5, 0, '17:45']]],
  [6,
   '00',
   870,
   [[0, 0, 7, '06:30'],
    [1, 5, 7, '08:55'],
    [2, 5, 0, '10:50'],
    [3, 6, 0, '12:40'],
    [4, 6, 0, '16:50']

# Stage 3: Modified List Scheduling for shifting the crew

In [ ]:
def work_shift(data,ns): #ns: maximum number of shifts for each week

    #Number of team
    nt = len(data) // ns + 1

    #initial data
    days = {}
    shifts = {}
    for t in range(nt):
        days[t] = []
        shifts[t] = []

    #data sorting
    data = copy.deepcopy(data)
    sorted_data = sorted(data, key=itemgetter(2,0),reverse = True)

    myshift = [[] for t in range(nt)]
    time = [sum(k[2] for k in myshift[t] if myshift[t] != []) for t in range(nt)]
    sorted_team = list(np.argsort(time))

    for shift in sorted_data:
        for t in sorted_team:
            if shift[0] in days[t]:
                continue
            elif len(days[t]) < ns:
                days[t].append(shift[0])
                shifts[t].append(shift)
                time[t] += shift[2]
                sorted_team = list(np.argsort(time))
                break
            else:
                continue

    for shift in shifts.keys(): #sort the shifts days in the shifts data
        shifts[shift] = sorted(shifts[shift], key=itemgetter(0))

    for team in days.keys(): #sort the shifts days
        days[team] = sorted(days[team])

    new_time = {} #change time data from list to dictionary
    for day in range(len(time)):
        new_time[day] = time[day]

    return shifts, days, new_time

In [ ]:
# Crew Scheduling for each home base (0: Adelaide, 1: Brisbane, 5: Melbourne, 6: Perth, 7: Sydney) with ns = 5
homebase = {0,1,5,6,7} #from MIP Model result
ns = 5
crew_shifts = {}
for city in homebase:
    crew_shifts[city] = work_shift(totalshifts[city], ns)

In [ ]:
crew_shifts

{0: ({0: [[0,
     '01',
     510,
     [[0, 0, 7, '06:20'],
      [1, 0, 7, '08:45'],
      [2, 0, 5, '11:10'],
      [3, 0, 5, '13:00']]],
    [1, '00', 290, [[0, 0, 7, '05:25'], [1, 0, 7, '07:50']]],
    [3,
     '01',
     510,
     [[0, 0, 7, '06:20'],
      [1, 0, 7, '08:45'],
      [2, 0, 5, '11:10'],
      [3, 0, 5, '13:00']]],
    [4,
     '02',
     880,
     [[0, 0, 3, '05:00'],
      [1, 0, 3, '09:15'],
      [2, 0, 5, '13:30'],
      [3, 7, 5, '15:20'],
      [4, 7, 0, '17:15']]],
    [6,
     '01',
     720,
     [[0, 0, 3, '08:45'], [1, 1, 3, '13:00'], [2, 1, 0, '17:40']]]],
   1: [[1,
     '00',
     645,
     [[2, 0, 6, '10:15'], [3, 5, 6, '14:25'], [4, 5, 0, '19:10']]],
    [2,
     '01',
     335,
     [[4, 0, 5, '15:25'], [5, 2, 5, '17:15'], [6, 2, 0, '18:55']]],
    [3,
     '02',
     880,
     [[0, 0, 3, '05:00'],
      [1, 0, 3, '09:15'],
      [2, 0, 5, '13:30'],
      [3, 7, 5, '15:20'],
      [4, 7, 0, '17:15']]],
    [5,
     '00',
     645,
     [[0, 0, 6, 

## Adelaide Shifts

### Ideal working hour = 36 hours/ week = 2160 minutes/ week

In [ ]:
# Maximum number of shifts (ns): 5
work_shift(totalshifts[0], ns = 5)

({0: [[0,
    '01',
    510,
    [[0, 0, 7, '06:20'],
     [1, 0, 7, '08:45'],
     [2, 0, 5, '11:10'],
     [3, 0, 5, '13:00']]],
   [1, '00', 290, [[0, 0, 7, '05:25'], [1, 0, 7, '07:50']]],
   [3,
    '01',
    510,
    [[0, 0, 7, '06:20'],
     [1, 0, 7, '08:45'],
     [2, 0, 5, '11:10'],
     [3, 0, 5, '13:00']]],
   [4,
    '02',
    880,
    [[0, 0, 3, '05:00'],
     [1, 0, 3, '09:15'],
     [2, 0, 5, '13:30'],
     [3, 7, 5, '15:20'],
     [4, 7, 0, '17:15']]],
   [6,
    '01',
    720,
    [[0, 0, 3, '08:45'], [1, 1, 3, '13:00'], [2, 1, 0, '17:40']]]],
  1: [[1,
    '00',
    645,
    [[2, 0, 6, '10:15'], [3, 5, 6, '14:25'], [4, 5, 0, '19:10']]],
   [2,
    '01',
    335,
    [[4, 0, 5, '15:25'], [5, 2, 5, '17:15'], [6, 2, 0, '18:55']]],
   [3,
    '02',
    880,
    [[0, 0, 3, '05:00'],
     [1, 0, 3, '09:15'],
     [2, 0, 5, '13:30'],
     [3, 7, 5, '15:20'],
     [4, 7, 0, '17:15']]],
   [5,
    '00',
    645,
    [[0, 0, 6, '08:50'], [1, 5, 6, '13:00'], [2, 5, 0, '17:45']]]

In [ ]:
# Maximum number of shifts (ns): 4
work_shift(totalshifts[0], ns = 4)

({0: [[1, '00', 290, [[0, 0, 7, '05:25'], [1, 0, 7, '07:50']]],
   [2,
    '00',
    645,
    [[2, 0, 6, '10:15'], [3, 5, 6, '14:25'], [4, 5, 0, '19:10']]],
   [3,
    '01',
    510,
    [[0, 0, 7, '06:20'],
     [1, 0, 7, '08:45'],
     [2, 0, 5, '11:10'],
     [3, 0, 5, '13:00']]],
   [4,
    '02',
    880,
    [[0, 0, 3, '05:00'],
     [1, 0, 3, '09:15'],
     [2, 0, 5, '13:30'],
     [3, 7, 5, '15:20'],
     [4, 7, 0, '17:15']]]],
  1: [[0,
    '01',
    370,
    [[4, 0, 7, '14:50'], [5, 5, 7, '17:15'], [6, 5, 0, '19:10']]],
   [1,
    '00',
    645,
    [[2, 0, 6, '10:15'], [3, 5, 6, '14:25'], [4, 5, 0, '19:10']]],
   [3,
    '02',
    880,
    [[0, 0, 3, '05:00'],
     [1, 0, 3, '09:15'],
     [2, 0, 5, '13:30'],
     [3, 7, 5, '15:20'],
     [4, 7, 0, '17:15']]],
   [4,
    '01',
    370,
    [[4, 0, 7, '14:50'], [5, 5, 7, '17:15'], [6, 5, 0, '19:10']]]],
  2: [[0,
    '02',
    880,
    [[0, 0, 3, '05:00'],
     [1, 0, 3, '09:15'],
     [2, 0, 5, '13:30'],
     [3, 7, 5, '15:20

## Brisbane shifts

In [ ]:
# Maximum number of shifts (ns): 5
work_shift(totalshifts[1], ns = 5)

({0: [[0, '10', 340, [[4, 1, 5, '15:20'], [5, 1, 5, '18:10']]],
   [1,
    '10',
    410,
    [[0, 1, 7, '07:30'], [1, 5, 7, '09:35'], [2, 5, 1, '11:30']]],
   [2,
    '13',
    900,
    [[0, 1, 3, '06:00'], [1, 6, 3, '10:40'], [2, 6, 1, '15:00']]],
   [3,
    '10',
    550,
    [[0, 1, 5, '06:10'],
     [1, 0, 5, '09:00'],
     [2, 0, 7, '10:50'],
     [3, 1, 7, '13:15']]]],
  1: [[0,
    '10',
    550,
    [[0, 1, 5, '06:10'],
     [1, 0, 5, '09:00'],
     [2, 0, 7, '10:50'],
     [3, 1, 7, '13:15']]],
   [1,
    '13',
    900,
    [[0, 1, 3, '06:00'], [1, 6, 3, '10:40'], [2, 6, 1, '15:00']]],
   [2, '10', 400, [[3, 4, 1, '14:20'], [4, 4, 1, '17:40']]],
   [3, '10', 340, [[4, 1, 5, '15:20'], [5, 1, 5, '18:10']]],
   [6, '11', 250, [[0, 1, 7, '07:30'], [1, 1, 7, '09:35']]]],
  2: [[1, '11', 340, [[0, 1, 5, '07:20'], [1, 1, 5, '10:10']]],
   [2,
    '10',
    410,
    [[0, 1, 7, '07:30'], [1, 5, 7, '09:35'], [2, 5, 1, '11:30']]],
   [3,
    '12',
    355,
    [[4, 1, 7, '15:05'], [5, 2

In [ ]:
# Maximum number of shifts (ns): 4
work_shift(totalshifts[1], ns = 4)

({0: [[2,
    '13',
    900,
    [[0, 1, 3, '06:00'], [1, 6, 3, '10:40'], [2, 6, 1, '15:00']]],
   [4, '10', 340, [[4, 1, 5, '15:20'], [5, 1, 5, '18:10']]],
   [5,
    '12',
    535,
    [[3, 2, 1, '12:05'],
     [4, 2, 0, '14:25'],
     [5, 7, 0, '16:30'],
     [6, 7, 1, '18:55']]]],
  1: [[1,
    '13',
    900,
    [[0, 1, 3, '06:00'], [1, 6, 3, '10:40'], [2, 6, 1, '15:00']]],
   [2, '10', 400, [[3, 4, 1, '14:20'], [4, 4, 1, '17:40']]],
   [5,
    '10',
    455,
    [[0, 1, 7, '06:35'], [1, 0, 7, '08:40'], [2, 0, 1, '11:05']]],
   [6, '11', 250, [[0, 1, 7, '07:30'], [1, 1, 7, '09:35']]]],
  2: [[0, '13', 340, [[4, 1, 5, '15:20'], [5, 1, 5, '18:10']]],
   [1, '10', 400, [[3, 4, 1, '14:20'], [4, 4, 1, '17:40']]],
   [2,
    '11',
    480,
    [[2, 1, 7, '13:00'],
     [3, 5, 7, '15:05'],
     [4, 5, 2, '17:00'],
     [5, 1, 2, '18:40']]],
   [5,
    '11',
    825,
    [[0, 1, 0, '06:50'],
     [1, 1, 0, '09:55'],
     [2, 1, 0, '13:00'],
     [3, 7, 0, '16:05'],
     [4, 7, 1, '18:30']

## Melbourne shifts

In [ ]:
# Maximum number of shifts (ns): 5
work_shift(totalshifts[5], ns = 5)

({0: [[0, '54', 220, [[3, 0, 5, '17:15'], [4, 0, 5, '19:05']]],
   [3, '52', 220, [[4, 5, 0, '16:50'], [5, 5, 0, '18:40']]],
   [4,
    '52',
    570,
    [[0, 5, 1, '07:20'],
     [1, 5, 1, '10:10'],
     [2, 5, 7, '13:00'],
     [3, 5, 7, '14:55']]],
   [5,
    '55',
    960,
    [[0, 5, 7, '05:00'],
     [1, 1, 7, '06:55'],
     [2, 1, 6, '09:00'],
     [3, 0, 6, '15:00'],
     [4, 0, 5, '19:10']]],
   [6,
    '56',
    460,
    [[4, 5, 7, '13:20'],
     [5, 5, 7, '15:15'],
     [6, 5, 7, '17:10'],
     [7, 5, 7, '19:05']]]],
  1: [[0,
    '52',
    570,
    [[0, 5, 1, '07:20'],
     [1, 5, 1, '10:10'],
     [2, 5, 7, '13:00'],
     [3, 5, 7, '14:55']]],
   [1,
    '56',
    960,
    [[0, 5, 7, '05:00'],
     [1, 1, 7, '06:55'],
     [2, 1, 6, '09:00'],
     [3, 0, 6, '15:00'],
     [4, 0, 5, '19:10']]],
   [2, '50', 340, [[2, 5, 1, '15:20'], [3, 5, 1, '18:10']]],
   [3, '56', 200, [[2, 5, 2, '17:40'], [3, 5, 2, '19:20']]],
   [4,
    '53',
    410,
    [[3, 1, 5, '14:10'], [4, 1, 7

In [ ]:
# Maximum number of shifts (ns): 4
work_shift(totalshifts[5], ns = 4)

({0: [[0, '56', 200, [[2, 5, 2, '17:40'], [3, 5, 2, '19:20']]],
   [2,
    '51',
    480,
    [[0, 5, 7, '06:10'],
     [1, 1, 7, '08:05'],
     [2, 1, 7, '10:10'],
     [3, 5, 7, '12:15']]],
   [3,
    '55',
    360,
    [[3, 4, 5, '14:45'], [4, 4, 7, '16:30'], [5, 5, 7, '18:50']]],
   [5,
    '55',
    960,
    [[0, 5, 7, '05:00'],
     [1, 1, 7, '06:55'],
     [2, 1, 6, '09:00'],
     [3, 0, 6, '15:00'],
     [4, 0, 5, '19:10']]]],
  1: [[0, '54', 220, [[3, 0, 5, '17:15'], [4, 0, 5, '19:05']]],
   [2,
    '51',
    410,
    [[4, 5, 7, '14:10'], [5, 1, 7, '16:05'], [6, 1, 5, '18:10']]],
   [4, '51', 580, [[3, 3, 5, '11:20'], [4, 3, 5, '16:10']]],
   [6,
    '54',
    735,
    [[0, 5, 6, '05:00'], [1, 7, 6, '09:45'], [2, 7, 5, '15:20']]]],
  2: [[1,
    '51',
    410,
    [[4, 5, 7, '14:10'], [5, 1, 7, '16:05'], [6, 1, 5, '18:10']]],
   [2, '57', 220, [[0, 5, 0, '05:05'], [1, 5, 0, '06:55']]],
   [3, '51', 580, [[3, 3, 5, '11:20'], [4, 3, 5, '16:10']]],
   [5,
    '51',
    735,
    [

## Perth shifts

In [ ]:
# Maximum number of shifts (ns): 5
work_shift(totalshifts[6], ns = 5)

({0: [[0,
    '61',
    940,
    [[0, 6, 5, '05:20'],
     [1, 7, 5, '10:05'],
     [2, 7, 0, '12:00'],
     [3, 5, 0, '14:25'],
     [4, 5, 6, '16:15']]],
   [1,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 7, '12:15'],
     [3, 5, 7, '14:20'],
     [4, 5, 6, '16:15']]],
   [4,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]],
   [6,
    '60',
    820,
    [[0, 6, 1, '07:00'], [1, 7, 1, '13:00'], [2, 7, 6, '15:05']]]],
  1: [[0,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]],
   [1,
    '61',
    900,
    [[0, 6, 3, '06:00'], [1, 1, 3, '10:20'], [2, 1, 6, '15:00']]],
   [2,
    '61',
    900,
    [[0, 6, 3, '06:00'], [1, 1, 3, '10:20'], [2, 1, 6, '15:00']]],
   [3,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
 

In [ ]:
# Maximum number of shifts (ns): 4
work_shift(totalshifts[6], ns = 4)

({0: [[0,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]],
   [1,
    '61',
    900,
    [[0, 6, 3, '06:00'], [1, 1, 3, '10:20'], [2, 1, 6, '15:00']]],
   [4,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]]],
  1: [[3,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]],
   [4,
    '61',
    940,
    [[0, 6, 5, '05:20'],
     [1, 7, 5, '10:05'],
     [2, 7, 0, '12:00'],
     [3, 5, 0, '14:25'],
     [4, 5, 6, '16:15']]],
   [5,
    '60',
    900,
    [[0, 6, 3, '06:00'], [1, 1, 3, '10:20'], [2, 1, 6, '15:00']]],
   [6,
    '60',
    820,
    [[0, 6, 1, '07:00'], [1, 7, 1, '13:00'], [2, 7, 6, '15:05']]]],
  2: [[2,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 7, '12:15'

In [ ]:
# Maximum number of shifts (ns): 3
work_shift(totalshifts[6], ns = 3)

({0: [[3,
    '61',
    940,
    [[0, 6, 5, '05:20'],
     [1, 7, 5, '10:05'],
     [2, 7, 0, '12:00'],
     [3, 5, 0, '14:25'],
     [4, 5, 6, '16:15']]],
   [4,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]],
   [6,
    '60',
    820,
    [[0, 6, 1, '07:00'], [1, 7, 1, '13:00'], [2, 7, 6, '15:05']]]],
  1: [[3,
    '60',
    960,
    [[0, 6, 0, '05:00'],
     [1, 1, 0, '09:10'],
     [2, 1, 2, '12:15'],
     [3, 5, 2, '14:35'],
     [4, 5, 6, '16:15']]],
   [4,
    '61',
    940,
    [[0, 6, 5, '05:20'],
     [1, 7, 5, '10:05'],
     [2, 7, 0, '12:00'],
     [3, 5, 0, '14:25'],
     [4, 5, 6, '16:15']]],
   [6,
    '61',
    815,
    [[0, 6, 5, '07:25'], [1, 1, 5, '12:10'], [2, 1, 6, '15:00']]]],
  2: [[0,
    '61',
    940,
    [[0, 6, 5, '05:20'],
     [1, 7, 5, '10:05'],
     [2, 7, 0, '12:00'],
     [3, 5, 0, '14:25'],
     [4, 5, 6, '16:15']]],
   [2,
    '60',
    960,
    [[0, 6, 

## Sydney shifts

In [ ]:
# Maximum number of shifts (ns): 5
work_shift(totalshifts[7], ns = 5)

({0: [[0,
    '72',
    360,
    [[0, 7, 4, '08:55'], [1, 5, 4, '11:15'], [2, 5, 7, '13:00']]],
   [1,
    '74',
    670,
    [[0, 7, 5, '06:00'],
     [1, 7, 5, '07:55'],
     [2, 7, 5, '09:50'],
     [3, 4, 5, '11:45'],
     [4, 4, 5, '13:30'],
     [5, 7, 5, '15:15']]],
   [3, '74', 230, [[0, 7, 5, '09:10'], [1, 7, 5, '11:05']]],
   [4, '71', 280, [[0, 7, 4, '05:10'], [1, 7, 4, '07:30']]],
   [6,
    '72',
    820,
    [[0, 7, 1, '07:20'], [1, 6, 1, '09:25'], [2, 6, 7, '15:25']]]],
  1: [[0, '70', 230, [[0, 7, 5, '05:25'], [1, 7, 5, '07:20']]],
   [3,
    '75',
    820,
    [[0, 7, 1, '07:20'], [1, 6, 1, '09:25'], [2, 6, 7, '15:25']]],
   [4, '74', 230, [[0, 7, 5, '09:10'], [1, 7, 5, '11:05']]],
   [5,
    '72',
    640,
    [[2, 7, 5, '10:20'],
     [3, 1, 5, '12:15'],
     [4, 1, 7, '15:05'],
     [5, 5, 7, '17:10'],
     [6, 5, 7, '19:05']]],
   [6,
    '71',
    460,
    [[0, 7, 5, '07:10'],
     [1, 7, 5, '09:05'],
     [2, 7, 5, '11:00'],
     [3, 7, 5, '12:55']]]],
  2: [[0, 

In [ ]:
# Maximum number of shifts (ns): 4
work_shift(totalshifts[7], ns = 4)

({0: [[0,
    '76',
    480,
    [[4, 7, 1, '13:00'],
     [5, 7, 1, '15:05'],
     [6, 7, 5, '17:10'],
     [7, 7, 5, '19:05']]],
   [4,
    '74',
    480,
    [[2, 7, 1, '13:00'],
     [3, 7, 1, '15:05'],
     [4, 7, 5, '17:10'],
     [5, 7, 5, '19:05']]],
   [6,
    '72',
    820,
    [[0, 7, 1, '07:20'], [1, 6, 1, '09:25'], [2, 6, 7, '15:25']]]],
  1: [[1, '75', 280, [[4, 7, 4, '16:20'], [5, 7, 4, '18:40']]],
   [2, '74', 230, [[6, 7, 5, '17:10'], [7, 7, 5, '19:05']]],
   [5,
    '72',
    640,
    [[2, 7, 5, '10:20'],
     [3, 1, 5, '12:15'],
     [4, 1, 7, '15:05'],
     [5, 5, 7, '17:10'],
     [6, 5, 7, '19:05']]],
   [6,
    '74',
    735,
    [[0, 7, 6, '05:00'], [1, 5, 6, '10:35'], [2, 5, 7, '15:20']]]],
  2: [[1, '70', 230, [[5, 5, 7, '17:10'], [6, 5, 7, '19:05']]],
   [2,
    '75',
    570,
    [[0, 7, 5, '06:50'],
     [1, 1, 5, '08:45'],
     [2, 1, 5, '11:35'],
     [3, 7, 5, '14:25']]],
   [4,
    '72',
    360,
    [[0, 7, 4, '08:55'], [1, 5, 4, '11:15'], [2, 5, 7, '1